<a href="https://colab.research.google.com/github/MasaHirai-Jinen/Medical_Institution_db_colab/blob/main/%E5%8C%BB%E7%99%82%E6%A9%9F%E9%96%A2%E5%8F%97%E7%90%86%E5%B1%8A%E9%A0%85%E7%9B%AE%E5%88%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
from google.colab import auth

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

zip_file_path = "jmidb-bucket.zip"
file_id = '1kGQtfh7ZZDHLeK-0_OpFubKod06zb9ZL' # https://drive.google.com/file/d/1kGQtfh7ZZDHLeK-0_OpFubKod06zb9ZL/view?usp=drive_link
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile(zip_file_path)

few_docor_path = "医師少数区域とスポットのICU.xlsx"
file_id = '1dxAQTJ_0my7uq4DRW-MtZErJ1PA4O_wP' # https://docs.google.com/spreadsheets/d/1dxAQTJ_0my7uq4DRW-MtZErJ1PA4O_wP/edit?usp=drive_link&ouid=117477706610962127029&rtpof=true&sd=true
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile(few_docor_path)

### 共有method

In [8]:
import pandas as pd
import re

In [9]:
def print_dataframe_info(df:pd.DataFrame):
    print(df.info())
    print('-'*100)
    display(df)

In [10]:
import zipfile
import io
from datetime import datetime
from typing import List, Tuple


def extract_files_from_zip (zip_filepath, code_string) -> List[Tuple[datetime, str, io.BytesIO]]:
    file_list = []
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        for info in zip_ref.infolist():
            l = info.filename.split('/')
            if (code_string in l) and (not info.is_dir()):
                i = l.index(code_string)
                category, kikan, download_date, *filename = l[i:]
                download_date = datetime.strptime(download_date, '%Y-%m-%d')
                with zip_ref.open(info) as file:
                    buffer = io.BytesIO(file.read())
                file_list.append((download_date, info.filename, buffer))

    file_list.sort(key=lambda x: x[0])
    return file_list

In [11]:
import pandas as pd
from typing import NamedTuple

TodofukenType = NamedTuple('TodofukenType', [('id', int), ('name', str)])

class TodofukenDataFrame:
    """
    都道府県データを保持するクラス

    Attributes:
        df (pd.DataFrame): 都道府県データ

    Examples:
        TODOFUKEN = TodofukenDataFrame()
        string = "ｘｘｘｘｘｘｘ kumamoto みえｘｘｘｘｘｘｘｘｘ"
        TODOFUKEN.find_todofuken_from_string(string) # -> TodofukenType(id=24, name='三重県')
    """
    __df: pd.DataFrame = pd.DataFrame(
            [[1,"北海道","北海道", "ほっかいどう", "hokkaido"],
            [2,"青森県","青森","あおもり", "aomori"],
            [3,"岩手県","岩手","いしかわ", "iwate"],
            [4,"宮城県","宮城","みやぎ", "miyagi"],
            [5,"秋田県","秋田","あきた", "akita"],
            [6,"山形県","山形","やまがた", "yamagata"],
            [7,"福島県","福島","ふくい", "fukushima"],
            [8,"茨城県","茨城","いばらき", "ibaraki"],
            [9,"栃木県","栃木","とちぎ", "tochigi"],
            [10,"群馬県","群馬","ぐんま", "gunma"],
            [11,"埼玉県","埼玉","さいたま", "saitama"],
            [12,"千葉県","千葉","ちば", "chiba"],
            [13,"東京都","東京","とうきょう", "tokyo"],
            [14,"神奈川県","神奈川","かながわ", "kanagawa"],
            [15,"新潟県","新潟","にいがた", "niigata"],
            [16,"富山県","富山","とやま", "toyama"],
            [17,"石川県","石川","いしかわ", "ishikawa"],
            [18,"福井県","福井","ふくい", "fukui"],
            [19,"山梨県","山梨","やまなし", "yamanashi"],
            [20,"長野県","長野","ながの", "nagano"],
            [21,"岐阜県","岐阜","ぎふ", "gifu"],
            [22,"静岡県","静岡","しずおか", "shizuoka"],
            [23,"愛知県","愛知","あいち", "aichi"],
            [24,"三重県","三重","みえ", "mie"],
            [25,"滋賀県","滋賀","しが", "shiga"],
            [26,"京都府","京都","きょうと", "kyoto"],
            [27,"大阪府","大阪","おおさか", "osaka"],
            [28,"兵庫県","兵庫","ひょうご", "hyogo"],
            [29,"奈良県","奈良","なら", "nara"],
            [30,"和歌山県","和歌山","わかやま", "wakayama"],
            [31,"鳥取県","鳥取","とっとり", "tottori"],
            [32,"島根県","島根", "しまね", "shimane"],
            [33,"岡山県","岡山","おかやま", "okayama"],
            [34,"広島県","広島","ひろしま", "hiroshima"],
            [35,"山口県","山口","やまぐち", "yamaguchi"],
            [36,"徳島県","徳島","とくしま", "tokushima"],
            [37,"香川県","香川","かがわ", "kagawa"],
            [38,"愛媛県","愛媛","えひめ", "ehime"],
            [39,"高知県","高知","こうち", "kochi"],
            [40,"福岡県","福岡","ふくおか", "fukuoka"],
            [41,"佐賀県","佐賀","さが", "saga"],
            [42,"長崎県","長崎","ながさき", "nagasaki"],
            [43,"熊本県","熊本","くまもと", "kumamoto"],
            [44,"大分県","大分","おおいた", "oita"],
            [45,"宮崎県","宮崎","みやざき", "miyazaki"],
            [46,"鹿児島県","鹿児島","かごしま", "kagoshima"],
            [47,"沖縄県","沖縄","おきなわ", "okinawa"]]
            , columns=["都道府県ID", "都道府県名", "漢字", "ひらがな", "英字"]
        )

    @property
    def df(self) -> pd.DataFrame:
        return self.__df

    def find_todofuken_from_string(self, string:str) -> TodofukenType|None:
        """
        都道府県名を検索する.
        注意：idが小さいものから順に検索される.

        Args:
            string (str): 検索する文字列

        Returns:
            TodofukenType|None: 検索結果
        """
        string = string.lower()
        mask = self.df[['漢字', 'ひらがな', '英字']].apply(lambda row: any(s in string for s in row), axis=1)
        if mask.any():
            row = self.df[mask].iloc[0]
            return TodofukenType(id = row["都道府県ID"], name = row["都道府県名"])
        return None

TODOFUKEN = TodofukenDataFrame()

In [12]:
import pandas as pd
from typing import NamedTuple

SyubetuType = NamedTuple('SyubetuType', [('id', int), ('name', str)])

class SyubetuData:
    __df: pd.DataFrame = pd.DataFrame(
        [[1, "医科"],
        [3, "歯科"],
        [4, "薬局"],
        ],
        columns=["種別ID", "種別名"]
    )

    @property
    def df(self) -> pd.DataFrame:
        return self.__df

    def find_syubetu_from_id(self, id:int) -> SyubetuType|None:
        if id in self.__df["種別ID"].values:
            return SyubetuType(id=id, name=self.__df[self.__df["種別ID"] == id]["種別名"].values[0])
        return None

    def find_syubetu_from_name(self, name:str) -> SyubetuType|None:
        if name in self.__df["種別名"].values:
            return SyubetuType(id=self.__df[self.__df["種別名"] == name]["種別ID"].values[0], name=name)
        return None

SYUBETU = SyubetuData()

In [13]:
from datetime import datetime
import re

def wareki_to_seireki(input_string:str, format_string:str) -> datetime:
    """
    和暦を西暦に変換する

    %g: 年号
    %e: 年
    %m: 月
    %d: 日

    Args:
        input_string  (str): 変換する文字列
        format_string (str): 変換する文字列のフォーマット

    Returns:
        datetime: 変換後の年月日

    Usage Example:
        wareki_to_seireki('昭和元年2月28日', '%g%e年%m月%d日') #-> datetime(1926, 2, 28, 0, 0)
    """
    ERA_DICT = {
        '令和': datetime(2019, 5, 1),
        '平成': datetime(1989, 1, 8),
        '昭和': datetime(1926, 12, 25),
        '大正': datetime(1912, 7, 30),
        '明治': datetime(1868, 1, 25),
    }

    # 正規表現のパターンを作成
    pattern = format_string.replace(r'%g', f"(?P<ear_name>{'|'.join(ERA_DICT.keys())})")
    pattern = pattern.replace(r'%e', r'(?P<ear_year>\d{1,2}|元)')
    pattern = pattern.replace(r'%m', r'(?P<month>\d{1,2})')
    pattern = pattern.replace(r'%d', r'(?P<day>\d{1,2})')

    # 正規表現でマッチ
    match = re.match(pattern, input_string)

    if match is None:
        raise ValueError(f"time data '{input_string}' does not match format '{format_string}'")

    ear_name = match.group('ear_name')
    ear_year = match.group('ear_year')

    if ear_name not in ERA_DICT.keys():
        raise ValueError(f"time data '{input_string}' does not match format '{format_string}'")

    if ear_year == '元':
        ear_year = 0
    else:
        ear_year = int(ear_year)-1
        if ear_year < 0:
            raise ValueError(f"time data '{input_string}' does not match format '{format_string}'")
    year = ERA_DICT[ear_name].year + ear_year

    # 新しいformat_stringを作成
    new_format_string = format_string.replace('%g', '').replace('%e', '')
    new_format_string+= "-%Y"

    # マッチした個所を削除したinput_stringを作成
    new_string = match.group(0)

    start , end = match.span(1)
    new_string = new_string[:start] + new_string[end:]
    i = end - start

    start , end = match.span(2)
    new_string = new_string[:start-i] + new_string[end-i:]

    new_string+= f"-{year}"

    return datetime.strptime(new_string, new_format_string)

### 個別

In [14]:
file_list = extract_files_from_zip(zip_file_path, 'accept_eath')
file_list[:5]

[(datetime.datetime(2024, 5, 17, 0, 0),
  'jmidb-bucket/accept_eath/tokaihokuriku/2024-05-17/aichi/2405\u3000届出受理医療機関名簿（届出項目別）/2405\u300001届出受理医療機関名簿（届出項目別）/2405（愛知）届出受理医療機関名簿（届出項目別）.xlsx',
  <_io.BytesIO at 0x798bc6efb100>),
 (datetime.datetime(2024, 5, 17, 0, 0),
  'jmidb-bucket/accept_eath/tokaihokuriku/2024-05-17/aichi/2405\u3000届出受理医療機関名簿（届出項目別）/2405\u300002届出受理医療機関名簿（届出項目別）/2405（愛知）届出受理医療機関名簿（届出項目別）.xlsx',
  <_io.BytesIO at 0x798bc6efb1f0>),
 (datetime.datetime(2024, 5, 17, 0, 0),
  'jmidb-bucket/accept_eath/tokaihokuriku/2024-05-17/aichi/2405\u3000届出受理医療機関名簿（届出項目別）/2405\u300003届出受理医療機関名簿（届出項目別）/2405（愛知）届出受理医療機関名簿（届出項目別）.xlsx',
  <_io.BytesIO at 0x798bc6efb150>),
 (datetime.datetime(2024, 5, 17, 0, 0),
  'jmidb-bucket/accept_eath/tokaihokuriku/2024-05-17/aichi/2405\u3000届出受理医療機関名簿（届出項目別）/2405\u300004届出受理医療機関名簿（届出項目別）/2405（愛知）届出受理医療機関名簿（届出項目別）.xlsx',
  <_io.BytesIO at 0x798bc6efb0b0>),
 (datetime.datetime(2024, 5, 17, 0, 0),
  'jmidb-bucket/accept_eath/tokaihokuriku/2024-05-17/aic

In [15]:
# 特定集中治療室管理料があるファイルのインデックス
[i for i, file_item in enumerate(file_list) if 'tokutei1' in file_item[1]]

[59, 274, 577, 776, 980, 1115]

In [16]:
download_date, filename, buffer = file_list[36]
dfs = pd.read_excel(buffer, sheet_name=None, dtype=str)

In [17]:
def download_excel(buffer):
    # bufferをローカルにdownload
    from google.colab import files
    buffer.seek(0)
    temp_file_name = "temp_file.xlsx"
    with open(temp_file_name, 'wb') as f:
        f.write(buffer.read())

    files.download(temp_file_name)

# download_excel(buffer)

In [18]:
sheet_name,df = list(dfs.items())[0]

def _crean_excel(df):
    df = df.copy()
    df.dropna(axis=0, how='all', inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

df = _crean_excel(df)
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,[令和 6年 5月 1日 現在 医科] 令和６年５月１６日作成,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,項番,都道府県コード,都道府県名,受理届出名称,医療機関番号,併設医療機関番号,医療機関記号番号,医療機関名称,医療機関所在地（郵便番号）,医療機関所在地（住所）,電話番号,FAX番号,病床数,受理記号,受理番号,算定開始年月日,個別有効開始年月日,備考（見出し）,備考（データ）
2,1,40,福岡県,精神科救急急性期医療入院料,0212514,0337846,NaN,雁の巣病院,811－0206,福岡県福岡市東区雁の巣１丁目２６番１号,092-606-2861,NaN,一般 29\n精神 264,精救,第6号,令和 4年10月 1日,NaN,NaN,NaN
3,1,40,福岡県,精神科救急急性期医療入院料,0212514,0337846,NaN,雁の巣病院,811－0206,福岡県福岡市東区雁の巣１丁目２６番１号,092-606-2861,NaN,一般 29\n精神 264,精救,第6号,令和 4年10月 1日,NaN,病棟数:,1棟
4,1,40,福岡県,精神科救急急性期医療入院料,0212514,0337846,NaN,雁の巣病院,811－0206,福岡県福岡市東区雁の巣１丁目２６番１号,092-606-2861,NaN,一般 29\n精神 264,精救,第6号,令和 4年10月 1日,NaN,病床数:,48床


In [19]:
# メタ情報ごとに分割
def _split_excel(df):
    df = df.copy()
    split_index = df.index[df.iloc[:,0].str.contains(r'\[.*\]', regex=True, na=False)]
    tantai_dfs = []
    for i, index in enumerate(split_index):
        block_df = df.iloc[index:split_index[i+1]].copy() if i < len(split_index)-1 else df.iloc[index:].copy()
        tantai_dfs.append(block_df)
    return tantai_dfs

tantai_dfs = _split_excel(df)
tantai_dfs[2].head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
942,[令和 6年 5月 1日 現在 医科] 令和６年５月１６日作成,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
943,項番,都道府県コード,都道府県名,受理届出名称,医療機関番号,併設医療機関番号,医療機関記号番号,医療機関名称,医療機関所在地（郵便番号）,医療機関所在地（住所）,電話番号,FAX番号,病床数,受理記号,受理番号,算定開始年月日,個別有効開始年月日,備考（見出し）,備考（データ）
944,1,42,長崎県,精神科救急急性期医療入院料,8124537,8133922,NaN,長崎県精神医療センター,856－0847,大村市西部町１５７５番地２,0957-53-3103,0957-52-2401,精神 139,精救,第2号,令和 4年 4月 1日,NaN,NaN,NaN
945,1,42,長崎県,精神科救急急性期医療入院料,8124537,8133922,NaN,長崎県精神医療センター,856－0847,大村市西部町１５７５番地２,0957-53-3103,0957-52-2401,精神 139,精救,第2号,令和 4年 4月 1日,NaN,病棟数:,1棟
946,1,42,長崎県,精神科救急急性期医療入院料,8124537,8133922,NaN,長崎県精神医療センター,856－0847,大村市西部町１５７５番地２,0957-53-3103,0957-52-2401,精神 139,精救,第2号,令和 4年 4月 1日,NaN,病床数:,47床


In [20]:
# メタ情報とデータの分割
tantai_df = tantai_dfs[2]

def _split_meta_and_data(df):
    df = df.copy()
    i = (df.iloc[:,0]=='項番	').idxmax()
    meta_df = df.loc[:i].copy()
    data = df.loc[i+1:].copy()
    return meta_df, data

meta_df, data = _split_meta_and_data(tantai_df)
meta_df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
942,[令和 6年 5月 1日 現在 医科] 令和６年５月１６日作成,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# メタデータの加工

def _extract_meta_info(df):
    df = df.copy()
    df.dropna(axis=0, how='all', inplace=True)
    df.dropna(axis=1, how='all', inplace=True)

    meta_info = df.apply(lambda x: x.str.cat(sep='\t'), axis=1)
    meta_info = meta_info.str.cat(sep='\n')

    def _extract_date_and_syubetu(meta_info):
        # 種別の抽出
        syubetu = None
        as_of_date = None
        created_date = None

        pattern = re.compile(r'\[(.*)現在\s(.*)\](.*)作成')
        match_ = re.search(pattern, meta_info)
        if match_:
            as_of_date, syubetu_name, created_date = match_.groups()
            syubetu_name = syubetu_name.strip()
            syubetu = SYUBETU.find_syubetu_from_name(syubetu_name)
        return syubetu, as_of_date, created_date

    syubetu, as_of_date, created_date = _extract_date_and_syubetu(meta_info)

    return meta_info, syubetu, as_of_date, created_date

meta_info, syubetu, as_of_date, created_date = _extract_meta_info(meta_df)
print(meta_info)
print(syubetu, as_of_date, created_date)

[令和 6年 5月 1日 現在　医科]　令和６年５月１６日作成
SyubetuType(id=1, name='医科') 令和 6年 5月 1日  　令和６年５月１６日


In [22]:
# データの加工
def _clean_data(df, as_of_date, created_date, syubetu):
    df = df.copy()

    df.columns = df.iloc[0]
    df = df[1:]
    df.reset_index(drop=True, inplace=True)
    df['基準日'] = as_of_date
    df['作成日'] = created_date
    df['点数区分コード'] = syubetu.id if syubetu else None
    df = df.astype({
        '都道府県コード': pd.Int64Dtype(),
        '点数区分コード': pd.Int64Dtype(),
        '医療機関番号': pd.Int64Dtype(),
        '併設医療機関番号': pd.Int64Dtype()})

    return df

data_df = _clean_data(data, as_of_date, created_date, syubetu)
print_dataframe_info(data_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338 entries, 0 to 337
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   項番             338 non-null    object
 1   都道府県コード        338 non-null    Int64 
 2   都道府県名          338 non-null    object
 3   受理届出名称         338 non-null    object
 4   医療機関番号         338 non-null    Int64 
 5   併設医療機関番号       99 non-null     Int64 
 6   医療機関記号番号       320 non-null    object
 7   医療機関名称         338 non-null    object
 8   医療機関所在地（郵便番号）  338 non-null    object
 9   医療機関所在地（住所）    338 non-null    object
 10  電話番号           338 non-null    object
 11  FAX番号          338 non-null    object
 12  病床数            338 non-null    object
 13  受理記号           338 non-null    object
 14  受理番号           338 non-null    object
 15  算定開始年月日        338 non-null    object
 16  個別有効開始年月日      0 non-null      object
 17  備考（見出し）        289 non-null    object
 18  備考（データ）        290 non-null   

943,項番,都道府県コード,都道府県名,受理届出名称,医療機関番号,併設医療機関番号,医療機関記号番号,医療機関名称,医療機関所在地（郵便番号）,医療機関所在地（住所）,...,病床数,受理記号,受理番号,算定開始年月日,個別有効開始年月日,備考（見出し）,備考（データ）,基準日,作成日,点数区分コード
0,1,42,長崎県,精神科救急急性期医療入院料,8124537,8133922,NaN,長崎県精神医療センター,856－0847,大村市西部町１５７５番地２,...,精神 139,精救,第2号,令和 4年 4月 1日,NaN,NaN,NaN,令和 6年 5月 1日,令和６年５月１６日,1
1,1,42,長崎県,精神科救急急性期医療入院料,8124537,8133922,NaN,長崎県精神医療センター,856－0847,大村市西部町１５７５番地２,...,精神 139,精救,第2号,令和 4年 4月 1日,NaN,病棟数:,1棟,令和 6年 5月 1日,令和６年５月１６日,1
2,1,42,長崎県,精神科救急急性期医療入院料,8124537,8133922,NaN,長崎県精神医療センター,856－0847,大村市西部町１５７５番地２,...,精神 139,精救,第2号,令和 4年 4月 1日,NaN,病床数:,47床,令和 6年 5月 1日,令和６年５月１６日,1
3,1,42,長崎県,精神科救急急性期医療入院料,8124537,8133922,NaN,長崎県精神医療センター,856－0847,大村市西部町１５７５番地２,...,精神 139,精救,第2号,令和 4年 4月 1日,NaN,看護職員夜間配置加算（精神科救急急性期医療入院料の注５）:,有,令和 6年 5月 1日,令和６年５月１６日,1
4,1,42,長崎県,精神科救急急性期医療入院料,8124537,8133922,NaN,長崎県精神医療センター,856－0847,大村市西部町１５７５番地２,...,精神 139,精救,第2号,令和 4年 4月 1日,NaN,NaN,精神科救急医療体制加算２,令和 6年 5月 1日,令和６年５月１６日,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,12,42,長崎県,認知症治療病棟入院料１,1220852,<NA>,医東85,医療法人さざなみ 鈴木病院,859－3807,東彼杵郡東彼杵町彼杵宿郷１０８５,...,療養 85\n精神 101,認治１,第8号,平成24年 8月 1日,NaN,病棟種別:,精神,令和 6年 5月 1日,令和６年５月１６日,1
334,12,42,長崎県,認知症治療病棟入院料１,1220852,<NA>,医東85,医療法人さざなみ 鈴木病院,859－3807,東彼杵郡東彼杵町彼杵宿郷１０８５,...,療養 85\n精神 101,認治１,第8号,平成24年 8月 1日,NaN,病棟数:,1,令和 6年 5月 1日,令和６年５月１６日,1
335,12,42,長崎県,認知症治療病棟入院料１,1220852,<NA>,医東85,医療法人さざなみ 鈴木病院,859－3807,東彼杵郡東彼杵町彼杵宿郷１０８５,...,療養 85\n精神 101,認治１,第8号,平成24年 8月 1日,NaN,病床数:,41,令和 6年 5月 1日,令和６年５月１６日,1
336,12,42,長崎県,認知症治療病棟入院料１,1220852,<NA>,医東85,医療法人さざなみ 鈴木病院,859－3807,東彼杵郡東彼杵町彼杵宿郷１０８５,...,療養 85\n精神 101,認治１,第8号,平成24年 8月 1日,NaN,退院調整加算:,無,令和 6年 5月 1日,令和６年５月１６日,1


In [23]:
# ベッド数の抽出
def _extract_bed_num(df):
    df = df.copy()
    groups = df.groupby(['医療機関番号', '都道府県コード', '受理届出名称','基準日'])
    bed_num_rows = []
    for (number, todofuken_num, juri, created_date), group_df in groups:
        # if not juri.startswith('特定集中治療室管理料'):
        #     continue
        bed_nums = group_df[group_df['備考（見出し）'].str.contains('病床数', na=False)]['備考（データ）']
        if not bed_nums.empty:
            bed_nums = bed_nums.str.extract(r'(\d+)').astype(int)
            bed_nums = bed_nums.iloc[:,0].values.tolist()
            for bed_num in bed_nums:
                bed_num_rows.append([number, todofuken_num, juri, created_date, bed_num])

    bed_num_df = pd.DataFrame(bed_num_rows, columns=['医療機関番号', '都道府県コード', '受理届出名称','基準日', 'ベッド数'])
    return bed_num_df

bed_num_df = _extract_bed_num(data_df)
print_dataframe_info(bed_num_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   医療機関番号   65 non-null     int64 
 1   都道府県コード  65 non-null     int64 
 2   受理届出名称   65 non-null     object
 3   基準日      65 non-null     object
 4   ベッド数     65 non-null     int64 
dtypes: int64(3), object(2)
memory usage: 2.7+ KB
None
----------------------------------------------------------------------------------------------------


,医療機関番号,都道府県コード,受理届出名称,基準日,ベッド数
0,112647,42,精神療養病棟入院料,令和 6年 5月 1日,60
1,112647,42,精神療養病棟入院料,令和 6年 5月 1日,60
2,112647,42,精神科急性期治療病棟入院料１,令和 6年 5月 1日,49
3,112753,42,精神療養病棟入院料,令和 6年 5月 1日,60
4,112753,42,精神療養病棟入院料,令和 6年 5月 1日,60
...,...,...,...,...,...
60,1510880,42,精神療養病棟入院料,令和 6年 5月 1日,60
61,1510880,42,精神療養病棟入院料,令和 6年 5月 1日,60
62,1521630,42,精神療養病棟入院料,令和 6年 5月 1日,60
63,1521630,42,精神療養病棟入院料,令和 6年 5月 1日,59


In [24]:
def _clean_data_df(df):
    df = df.copy()
    df.drop_duplicates(subset=['医療機関番号', '都道府県コード', '受理届出名称'], inplace=True)
    df.drop(columns=['項番','病床数','備考（見出し）','備考（データ）'], inplace=True)
    df.sort_values(by=['都道府県コード', '医療機関番号'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

resalt_df = _clean_data_df(data_df)
print_dataframe_info(resalt_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   都道府県コード        48 non-null     Int64 
 1   都道府県名          48 non-null     object
 2   受理届出名称         48 non-null     object
 3   医療機関番号         48 non-null     Int64 
 4   併設医療機関番号       11 non-null     Int64 
 5   医療機関記号番号       43 non-null     object
 6   医療機関名称         48 non-null     object
 7   医療機関所在地（郵便番号）  48 non-null     object
 8   医療機関所在地（住所）    48 non-null     object
 9   電話番号           48 non-null     object
 10  FAX番号          48 non-null     object
 11  受理記号           48 non-null     object
 12  受理番号           48 non-null     object
 13  算定開始年月日        48 non-null     object
 14  個別有効開始年月日      0 non-null      object
 15  基準日            48 non-null     object
 16  作成日            48 non-null     object
 17  点数区分コード        48 non-null     Int64 
dtypes: Int64(4), object(14)
memory u

943,都道府県コード,都道府県名,受理届出名称,医療機関番号,併設医療機関番号,医療機関記号番号,医療機関名称,医療機関所在地（郵便番号）,医療機関所在地（住所）,電話番号,FAX番号,受理記号,受理番号,算定開始年月日,個別有効開始年月日,基準日,作成日,点数区分コード
0,42,長崎県,精神科急性期治療病棟入院料１,112647,<NA>,医長264,医療法人志仁会 西脇病院,850－0835,長崎市桜木町３－１４,095-827-1187,095-822-8336,精急１,第10号,令和 2年12月 1日,NaN,令和 6年 5月 1日,令和６年５月１６日,1
1,42,長崎県,精神療養病棟入院料,112647,<NA>,医長264,医療法人志仁会 西脇病院,850－0835,長崎市桜木町３－１４,095-827-1187,095-822-8336,精療,第8号,令和 2年12月 1日,NaN,令和 6年 5月 1日,令和６年５月１６日,1
2,42,長崎県,精神科急性期治療病棟入院料１,112753,132757,医長275,医療法人厚生会 道ノ尾病院,852－8055,長崎市虹が丘町１－１,095-856-1111,095-856-4755,精急１,第4号,平成26年10月 1日,NaN,令和 6年 5月 1日,令和６年５月１６日,1
3,42,長崎県,精神療養病棟入院料,112753,132757,医長275,医療法人厚生会 道ノ尾病院,852－8055,長崎市虹が丘町１－１,095-856-1111,095-856-4755,精療,第2号,平成30年11月 1日,NaN,令和 6年 5月 1日,令和６年５月１６日,1
4,42,長崎県,精神科急性期治療病棟入院料１,121952,131957,医長195,医療法人友愛会 田川療養所,852－8045,長崎市錦２丁目１番１号,095-845-2188,095-844-0966,精急１,第11号,平成28年 7月 1日,NaN,令和 6年 5月 1日,令和６年５月１６日,1
5,42,長崎県,精神療養病棟入院料,121952,131957,医長195,医療法人友愛会 田川療養所,852－8045,長崎市錦２丁目１番１号,095-845-2188,095-844-0966,精療,第29号,平成28年 7月 1日,NaN,令和 6年 5月 1日,令和６年５月１６日,1
6,42,長崎県,精神療養病棟入院料,163442,<NA>,医長1344,医療法人昌生会 出口病院,851－1134,長崎市柿泊町２２５０番地,095-844-5293,095-842-1445,精療,第6号,平成26年 4月 1日,NaN,令和 6年 5月 1日,令和６年５月１６日,1
7,42,長崎県,認知症治療病棟入院料１,163442,<NA>,医長1344,医療法人昌生会 出口病院,851－1134,長崎市柿泊町２２５０番地,095-844-5293,095-842-1445,認治１,第11号,平成24年 8月 1日,NaN,令和 6年 5月 1日,令和６年５月１６日,1
8,42,長崎県,精神療養病棟入院料,164978,137020,医長1497,医療法人協治会 杠葉病院,850－0975,長崎市三和町４１３,095-878-3734,095-878-3289,精療,第24号,令和 6年 3月 1日,NaN,令和 6年 5月 1日,令和６年５月１６日,1
9,42,長崎県,認知症治療病棟入院料１,164978,137020,医長1497,医療法人協治会 杠葉病院,850－0975,長崎市三和町４１３,095-878-3734,095-878-3289,認治１,第7号,平成24年 6月 1日,NaN,令和 6年 5月 1日,令和６年５月１６日,1


In [25]:
def _merge_bed_num(df, bed_num_df):
    return df.merge(bed_num_df, on=['医療機関番号', '都道府県コード', '受理届出名称', '基準日'], how='left').astype({'ベッド数': pd.Int64Dtype()})

all_df = _merge_bed_num(resalt_df, bed_num_df)
print_dataframe_info(all_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   都道府県コード        69 non-null     Int64 
 1   都道府県名          69 non-null     object
 2   受理届出名称         69 non-null     object
 3   医療機関番号         69 non-null     Int64 
 4   併設医療機関番号       21 non-null     Int64 
 5   医療機関記号番号       64 non-null     object
 6   医療機関名称         69 non-null     object
 7   医療機関所在地（郵便番号）  69 non-null     object
 8   医療機関所在地（住所）    69 non-null     object
 9   電話番号           69 non-null     object
 10  FAX番号          69 non-null     object
 11  受理記号           69 non-null     object
 12  受理番号           69 non-null     object
 13  算定開始年月日        69 non-null     object
 14  個別有効開始年月日      0 non-null      object
 15  基準日            69 non-null     object
 16  作成日            69 non-null     object
 17  点数区分コード        69 non-null     Int64 
 18  ベッド数           65 non-null     I

,都道府県コード,都道府県名,受理届出名称,医療機関番号,併設医療機関番号,医療機関記号番号,医療機関名称,医療機関所在地（郵便番号）,医療機関所在地（住所）,電話番号,FAX番号,受理記号,受理番号,算定開始年月日,個別有効開始年月日,基準日,作成日,点数区分コード,ベッド数
0,42,長崎県,精神科急性期治療病棟入院料１,112647,<NA>,医長264,医療法人志仁会 西脇病院,850－0835,長崎市桜木町３－１４,095-827-1187,095-822-8336,精急１,第10号,令和 2年12月 1日,NaN,令和 6年 5月 1日,令和６年５月１６日,1,49
1,42,長崎県,精神療養病棟入院料,112647,<NA>,医長264,医療法人志仁会 西脇病院,850－0835,長崎市桜木町３－１４,095-827-1187,095-822-8336,精療,第8号,令和 2年12月 1日,NaN,令和 6年 5月 1日,令和６年５月１６日,1,60
2,42,長崎県,精神療養病棟入院料,112647,<NA>,医長264,医療法人志仁会 西脇病院,850－0835,長崎市桜木町３－１４,095-827-1187,095-822-8336,精療,第8号,令和 2年12月 1日,NaN,令和 6年 5月 1日,令和６年５月１６日,1,60
3,42,長崎県,精神科急性期治療病棟入院料１,112753,132757,医長275,医療法人厚生会 道ノ尾病院,852－8055,長崎市虹が丘町１－１,095-856-1111,095-856-4755,精急１,第4号,平成26年10月 1日,NaN,令和 6年 5月 1日,令和６年５月１６日,1,60
4,42,長崎県,精神療養病棟入院料,112753,132757,医長275,医療法人厚生会 道ノ尾病院,852－8055,長崎市虹が丘町１－１,095-856-1111,095-856-4755,精療,第2号,平成30年11月 1日,NaN,令和 6年 5月 1日,令和６年５月１６日,1,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,42,長崎県,精神療養病棟入院料,1510880,<NA>,医北松88,医療法人 敬仁会 松浦病院,859－6408,佐世保市世知原町栗迎９ー１,0956-76-2201,0956-76-2232,精療,第32号,平成26年 4月 1日,NaN,令和 6年 5月 1日,令和６年５月１６日,1,60
65,42,長崎県,精神療養病棟入院料,1521630,<NA>,医北松163,医療法人みなづき 佐々病院,857－0352,北松浦郡佐々町口石免１１０８－３,0956-62-2184,0956-62-6642,精療,第37号,平成26年 4月 1日,NaN,令和 6年 5月 1日,令和６年５月１６日,1,60
66,42,長崎県,精神療養病棟入院料,1521630,<NA>,医北松163,医療法人みなづき 佐々病院,857－0352,北松浦郡佐々町口石免１１０８－３,0956-62-2184,0956-62-6642,精療,第37号,平成26年 4月 1日,NaN,令和 6年 5月 1日,令和６年５月１６日,1,59
67,42,長崎県,精神科救急急性期医療入院料,8124537,8133922,NaN,長崎県精神医療センター,856－0847,大村市西部町１５７５番地２,0957-53-3103,0957-52-2401,精救,第2号,令和 4年 4月 1日,NaN,令和 6年 5月 1日,令和６年５月１６日,1,47


In [26]:
# 医師少数地域・スポット
def _create_few_doctors_df(xlsx_path):

    few_doctors_df = pd.read_excel(xlsx_path,dtype=str)
    few_doctors_df.set_index('#', drop=True, inplace=True)
    # indexがnanは削除
    few_doctors_df = few_doctors_df[few_doctors_df.index.notna()]
    # カラム名から※以降の文字列を削除
    few_doctors_df.columns = few_doctors_df.columns.str.replace(r'※.*', '', regex=True)
    # 医師少数区域列でnanをFalseそれ以外をTrueにする
    target_col = ['医師少数区域', '医師少数スポット','別表第六の二厚労大臣が定める地域']
    few_doctors_df[target_col] = few_doctors_df[target_col].notna()
    few_doctors_df['点数区分コード'] = 1
    few_doctors_df = few_doctors_df[['都道府県コード','医療機関番号','医師少数区域','医師少数スポット','別表第六の二厚労大臣が定める地域','点数区分コード']]
    few_doctors_df = few_doctors_df.astype({'都道府県コード': pd.Int64Dtype(), '医療機関番号': pd.Int64Dtype()})

    return few_doctors_df.copy()

few_doctors_df = _create_few_doctors_df(few_docor_path)
print_dataframe_info(few_doctors_df)
# few_doctors_df[['都道府県コード','点数区分コード','医療機関番号','医師少数区域','医師少数スポット','別表第六の二厚労大臣が定める地域']]

<class 'pandas.core.frame.DataFrame'>
Index: 29 entries, 1 to 29
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   都道府県コード           29 non-null     Int64
 1   医療機関番号            29 non-null     Int64
 2   医師少数区域            29 non-null     bool 
 3   医師少数スポット          29 non-null     bool 
 4   別表第六の二厚労大臣が定める地域  29 non-null     bool 
 5   点数区分コード           29 non-null     int64
dtypes: Int64(2), bool(3), int64(1)
memory usage: 1.0+ KB
None
----------------------------------------------------------------------------------------------------


,都道府県コード,医療機関番号,医師少数区域,医師少数スポット,別表第六の二厚労大臣が定める地域,点数区分コード
#,,,,,,
1,2,315255,True,False,False,1
2,2,810339,True,False,True,1
3,6,812497,True,False,False,1
4,7,211772,True,False,False,1
5,7,210246,True,False,False,1
6,7,411497,True,False,False,1
7,8,210047,True,False,False,1
8,8,3811171,True,False,False,1
9,8,810663,True,False,False,1


In [27]:
def _merge_few_doctors_df(all_df, few_doctors_df):
    return_df = all_df.merge(few_doctors_df, on=['都道府県コード', '点数区分コード', '医療機関番号'], how='left')

    bool_cols = ['医師少数区域', '医師少数スポット','別表第六の二厚労大臣が定める地域']
    return_df.loc[:,bool_cols] = return_df[bool_cols].astype(float).fillna(0).astype(bool)
    return return_df

return_df = _merge_few_doctors_df(all_df, few_doctors_df)
print_dataframe_info(return_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   都道府県コード           69 non-null     Int64 
 1   都道府県名             69 non-null     object
 2   受理届出名称            69 non-null     object
 3   医療機関番号            69 non-null     Int64 
 4   併設医療機関番号          21 non-null     Int64 
 5   医療機関記号番号          64 non-null     object
 6   医療機関名称            69 non-null     object
 7   医療機関所在地（郵便番号）     69 non-null     object
 8   医療機関所在地（住所）       69 non-null     object
 9   電話番号              69 non-null     object
 10  FAX番号             69 non-null     object
 11  受理記号              69 non-null     object
 12  受理番号              69 non-null     object
 13  算定開始年月日           69 non-null     object
 14  個別有効開始年月日         0 non-null      object
 15  基準日               69 non-null     object
 16  作成日               69 non-null     object
 17  点数区分コード           

,都道府県コード,都道府県名,受理届出名称,医療機関番号,併設医療機関番号,医療機関記号番号,医療機関名称,医療機関所在地（郵便番号）,医療機関所在地（住所）,電話番号,...,受理番号,算定開始年月日,個別有効開始年月日,基準日,作成日,点数区分コード,ベッド数,医師少数区域,医師少数スポット,別表第六の二厚労大臣が定める地域
0,42,長崎県,精神科急性期治療病棟入院料１,112647,<NA>,医長264,医療法人志仁会 西脇病院,850－0835,長崎市桜木町３－１４,095-827-1187,...,第10号,令和 2年12月 1日,NaN,令和 6年 5月 1日,令和６年５月１６日,1,49,False,False,False
1,42,長崎県,精神療養病棟入院料,112647,<NA>,医長264,医療法人志仁会 西脇病院,850－0835,長崎市桜木町３－１４,095-827-1187,...,第8号,令和 2年12月 1日,NaN,令和 6年 5月 1日,令和６年５月１６日,1,60,False,False,False
2,42,長崎県,精神療養病棟入院料,112647,<NA>,医長264,医療法人志仁会 西脇病院,850－0835,長崎市桜木町３－１４,095-827-1187,...,第8号,令和 2年12月 1日,NaN,令和 6年 5月 1日,令和６年５月１６日,1,60,False,False,False
3,42,長崎県,精神科急性期治療病棟入院料１,112753,132757,医長275,医療法人厚生会 道ノ尾病院,852－8055,長崎市虹が丘町１－１,095-856-1111,...,第4号,平成26年10月 1日,NaN,令和 6年 5月 1日,令和６年５月１６日,1,60,False,False,False
4,42,長崎県,精神療養病棟入院料,112753,132757,医長275,医療法人厚生会 道ノ尾病院,852－8055,長崎市虹が丘町１－１,095-856-1111,...,第2号,平成30年11月 1日,NaN,令和 6年 5月 1日,令和６年５月１６日,1,60,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,42,長崎県,精神療養病棟入院料,1510880,<NA>,医北松88,医療法人 敬仁会 松浦病院,859－6408,佐世保市世知原町栗迎９ー１,0956-76-2201,...,第32号,平成26年 4月 1日,NaN,令和 6年 5月 1日,令和６年５月１６日,1,60,False,False,False
65,42,長崎県,精神療養病棟入院料,1521630,<NA>,医北松163,医療法人みなづき 佐々病院,857－0352,北松浦郡佐々町口石免１１０８－３,0956-62-2184,...,第37号,平成26年 4月 1日,NaN,令和 6年 5月 1日,令和６年５月１６日,1,60,False,False,False
66,42,長崎県,精神療養病棟入院料,1521630,<NA>,医北松163,医療法人みなづき 佐々病院,857－0352,北松浦郡佐々町口石免１１０８－３,0956-62-2184,...,第37号,平成26年 4月 1日,NaN,令和 6年 5月 1日,令和６年５月１６日,1,59,False,False,False
67,42,長崎県,精神科救急急性期医療入院料,8124537,8133922,NaN,長崎県精神医療センター,856－0847,大村市西部町１５７５番地２,0957-53-3103,...,第2号,令和 4年 4月 1日,NaN,令和 6年 5月 1日,令和６年５月１６日,1,47,False,False,False


In [28]:
return_df[return_df[['医師少数区域', '医師少数スポット','別表第六の二厚労大臣が定める地域']].any(axis=1)]

,都道府県コード,都道府県名,受理届出名称,医療機関番号,併設医療機関番号,医療機関記号番号,医療機関名称,医療機関所在地（郵便番号）,医療機関所在地（住所）,電話番号,...,受理番号,算定開始年月日,個別有効開始年月日,基準日,作成日,点数区分コード,ベッド数,医師少数区域,医師少数スポット,別表第六の二厚労大臣が定める地域


### まとめ

In [29]:
from tqdm import tqdm

def main():
    file_list = extract_files_from_zip(zip_file_path, 'accept_eath')
    few_doctors_df = _create_few_doctors_df(few_docor_path)
    return_dfs = []
    for (download_date, filename, buffer) in tqdm(file_list):
        dfs = pd.read_excel(buffer, sheet_name=None, dtype=str)
        sheet_dfs = []

        for sheet_name, df in dfs.items():
            df = _crean_excel(df)
            tantai_dfs = _split_excel(df)
            for tantai_df in tantai_dfs:
                meta_df, data_df = _split_meta_and_data(tantai_df)
                meta_info, syubetu, as_of_date, created_date = _extract_meta_info(meta_df)
                cleaned_data_df = _clean_data(data_df, as_of_date, created_date, syubetu)
                bed_num_df = _extract_bed_num(cleaned_data_df)
                comped_data_df = _clean_data_df(cleaned_data_df)
                sheet_df = _merge_bed_num(comped_data_df, bed_num_df)

                sheet_dfs.append(sheet_df)
        file_df = pd.concat(sheet_dfs)

        file_df['ファイルパス'] = filename
        file_df['データ取得日'] = download_date
        return_df = _merge_few_doctors_df(file_df, few_doctors_df)
        return_dfs.append(return_df)
    return pd.concat(return_dfs)

main_df = main()
print_dataframe_info(main_df)

100%|██████████| 1121/1121 [22:17<00:00,  1.19s/it]


<class 'pandas.core.frame.DataFrame'>
Index: 909434 entries, 0 to 1287
Data columns (total 24 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   都道府県コード           909434 non-null  Int64         
 1   都道府県名             909434 non-null  object        
 2   受理届出名称            909434 non-null  object        
 3   医療機関番号            909434 non-null  Int64         
 4   併設医療機関番号          77403 non-null   Int64         
 5   医療機関記号番号          211598 non-null  object        
 6   医療機関名称            909434 non-null  object        
 7   医療機関所在地（郵便番号）     909434 non-null  object        
 8   医療機関所在地（住所）       909434 non-null  object        
 9   電話番号              909434 non-null  object        
 10  FAX番号             728216 non-null  object        
 11  受理記号              909434 non-null  object        
 12  受理番号              909434 non-null  object        
 13  算定開始年月日           909434 non-null  object        
 14  個別有効開始年月日  

,都道府県コード,都道府県名,受理届出名称,医療機関番号,併設医療機関番号,医療機関記号番号,医療機関名称,医療機関所在地（郵便番号）,医療機関所在地（住所）,電話番号,...,個別有効開始年月日,基準日,作成日,点数区分コード,ベッド数,ファイルパス,データ取得日,医師少数区域,医師少数スポット,別表第六の二厚労大臣が定める地域
0,23,愛知県,精神病棟入院基本料,100058,110053,千種5,愛知県精神医療センター,464－0031,名古屋市千種区徳川山町４－１－７,052-763-1511,...,NaN,令和 6年 4月 1日,令和６年５月１５日,1,145,jmidb-bucket/accept_eath/tokaihokuriku/2024-05...,2024-05-17,False,False,False
1,23,愛知県,特定機能病院入院基本料,101536,111531,千種153,愛知県がんセンター,464－0021,名古屋市千種区鹿子殿１－１,052-762-6111,...,NaN,令和 6年 4月 1日,令和６年５月１５日,1,492,jmidb-bucket/accept_eath/tokaihokuriku/2024-05...,2024-05-17,False,False,False
2,23,愛知県,一般病棟入院基本料,102088,<NA>,千種208,医療法人吉田病院,464－0854,名古屋市千種区大久手町５－１９,052-741-4187,...,NaN,令和 6年 4月 1日,令和６年５月１５日,1,57,jmidb-bucket/accept_eath/tokaihokuriku/2024-05...,2024-05-17,False,False,False
3,23,愛知県,療養病棟入院基本料,102088,<NA>,千種208,医療法人吉田病院,464－0854,名古屋市千種区大久手町５－１９,052-741-4187,...,NaN,令和 6年 4月 1日,令和６年５月１５日,1,51,jmidb-bucket/accept_eath/tokaihokuriku/2024-05...,2024-05-17,False,False,False
4,23,愛知県,一般病棟入院基本料,102377,<NA>,千種237,医療法人蜂友会 はちや整形外科病院,464－0821,名古屋市千種区末盛通２－４,052-751-8188,...,NaN,令和 6年 4月 1日,令和６年５月１５日,1,52,jmidb-bucket/accept_eath/tokaihokuriku/2024-05...,2024-05-17,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1283,30,和歌山県,短期滞在手術等基本料１,2310284,<NA>,新医1028,さとう眼科,647－0045,新宮市井の沢１２－６,0735-28-0310,...,NaN,令和 6年11月 1日,令和６年１１月１日,1,4,jmidb-bucket/accept_eath/kinki/2024-11-06/2024...,2024-11-06,False,False,False
1284,30,和歌山県,短期滞在手術等基本料１,2410803,<NA>,西医1080,上富田クリニック,649－2105,西牟婁郡上富田町朝来１４０７－１,0739-47-1100,...,NaN,令和 6年11月 1日,令和６年１１月１日,1,<NA>,jmidb-bucket/accept_eath/kinki/2024-11-06/2024...,2024-11-06,False,False,False
1285,30,和歌山県,短期滞在手術等基本料１,2510461,<NA>,東医1046,医療法人健佑会 串本有田病院,649－3514,東牟婁郡串本町有田４９９番地の１,0735-66-1021,...,NaN,令和 6年11月 1日,令和６年１１月１日,1,4,jmidb-bucket/accept_eath/kinki/2024-11-06/2024...,2024-11-06,False,False,False
1286,30,和歌山県,短期滞在手術等基本料１,2510479,<NA>,東医1047,串本リハビリテーションセンター,649－3503,東牟婁郡串本町串本２５９－６,0735-62-3600,...,NaN,令和 6年11月 1日,令和６年１１月１日,1,<NA>,jmidb-bucket/accept_eath/kinki/2024-11-06/2024...,2024-11-06,False,False,False


In [30]:
output_csv_path = 'main_df.csv'
main_df.to_csv(output_csv_path, index=False)

In [31]:
output_csv_path = 'main_df.csv'
main_df = pd.read_csv(output_csv_path, dtype=str)

In [32]:
%%time
def _astype(df:pd.DataFrame) -> pd.DataFrame:
    """
    型変換.
    対象カラム：
        都道府県コード: str -> int
        点数区分コード: str -> int
        医療機関番号: str -> int
        ベッド数: str -> int
        作成日: str -> datetime
        基準日: str -> datetime
        データ取得日: str -> datetime
        医師少数区域: str -> bool
        医師少数スポット: str -> bool
        別表第六の二厚労大臣が定める地域: str -> bool

    """
    df = df.copy()

    df = df.astype({
        '都道府県コード': pd.Int64Dtype(),
        '点数区分コード': pd.Int64Dtype(),
        '医療機関番号': pd.Int64Dtype(),
        'ベッド数': pd.Int64Dtype()
        })

    def _wareki_to_seireki(x):
        try:
            return wareki_to_seireki(x, '%g%e年%m月%d日') if x else pd.NaT
        except ValueError:
            return pd.NaT

    zenkaku_to_hankaku = str.maketrans({chr(0xFF01 + i): chr(0x21 + i) for i in range(94)})
    for col_name in ['作成日', '基準日']:
        df[col_name] = df[col_name].str.translate(zenkaku_to_hankaku)\
                                    .str.replace(r'\s', '', regex=True)\
                                    .map(_wareki_to_seireki)\
                                    .astype('datetime64[ns]')

    df['データ取得日'] = pd.to_datetime(df['データ取得日'])

    bool_col = ['医師少数区域', '医師少数スポット', '別表第六の二厚労大臣が定める地域']
    df[bool_col] = df[bool_col]=='True'

    return df

output_df = _astype(main_df)
print_dataframe_info(output_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 909434 entries, 0 to 909433
Data columns (total 24 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   都道府県コード           909434 non-null  Int64         
 1   都道府県名             909434 non-null  object        
 2   受理届出名称            909434 non-null  object        
 3   医療機関番号            909434 non-null  Int64         
 4   併設医療機関番号          77403 non-null   object        
 5   医療機関記号番号          211598 non-null  object        
 6   医療機関名称            909434 non-null  object        
 7   医療機関所在地（郵便番号）     909434 non-null  object        
 8   医療機関所在地（住所）       909434 non-null  object        
 9   電話番号              909434 non-null  object        
 10  FAX番号             728216 non-null  object        
 11  受理記号              909434 non-null  object        
 12  受理番号              909434 non-null  object        
 13  算定開始年月日           909434 non-null  object        
 14  個別有効

,都道府県コード,都道府県名,受理届出名称,医療機関番号,併設医療機関番号,医療機関記号番号,医療機関名称,医療機関所在地（郵便番号）,医療機関所在地（住所）,電話番号,...,個別有効開始年月日,基準日,作成日,点数区分コード,ベッド数,ファイルパス,データ取得日,医師少数区域,医師少数スポット,別表第六の二厚労大臣が定める地域
0,23,愛知県,精神病棟入院基本料,100058,110053,千種5,愛知県精神医療センター,464－0031,名古屋市千種区徳川山町４－１－７,052-763-1511,...,NaN,2024-04-01,2024-05-15,1,145,jmidb-bucket/accept_eath/tokaihokuriku/2024-05...,2024-05-17,False,False,False
1,23,愛知県,特定機能病院入院基本料,101536,111531,千種153,愛知県がんセンター,464－0021,名古屋市千種区鹿子殿１－１,052-762-6111,...,NaN,2024-04-01,2024-05-15,1,492,jmidb-bucket/accept_eath/tokaihokuriku/2024-05...,2024-05-17,False,False,False
2,23,愛知県,一般病棟入院基本料,102088,NaN,千種208,医療法人吉田病院,464－0854,名古屋市千種区大久手町５－１９,052-741-4187,...,NaN,2024-04-01,2024-05-15,1,57,jmidb-bucket/accept_eath/tokaihokuriku/2024-05...,2024-05-17,False,False,False
3,23,愛知県,療養病棟入院基本料,102088,NaN,千種208,医療法人吉田病院,464－0854,名古屋市千種区大久手町５－１９,052-741-4187,...,NaN,2024-04-01,2024-05-15,1,51,jmidb-bucket/accept_eath/tokaihokuriku/2024-05...,2024-05-17,False,False,False
4,23,愛知県,一般病棟入院基本料,102377,NaN,千種237,医療法人蜂友会 はちや整形外科病院,464－0821,名古屋市千種区末盛通２－４,052-751-8188,...,NaN,2024-04-01,2024-05-15,1,52,jmidb-bucket/accept_eath/tokaihokuriku/2024-05...,2024-05-17,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909429,30,和歌山県,短期滞在手術等基本料１,2310284,NaN,新医1028,さとう眼科,647－0045,新宮市井の沢１２－６,0735-28-0310,...,NaN,2024-11-01,2024-11-01,1,4,jmidb-bucket/accept_eath/kinki/2024-11-06/2024...,2024-11-06,False,False,False
909430,30,和歌山県,短期滞在手術等基本料１,2410803,NaN,西医1080,上富田クリニック,649－2105,西牟婁郡上富田町朝来１４０７－１,0739-47-1100,...,NaN,2024-11-01,2024-11-01,1,<NA>,jmidb-bucket/accept_eath/kinki/2024-11-06/2024...,2024-11-06,False,False,False
909431,30,和歌山県,短期滞在手術等基本料１,2510461,NaN,東医1046,医療法人健佑会 串本有田病院,649－3514,東牟婁郡串本町有田４９９番地の１,0735-66-1021,...,NaN,2024-11-01,2024-11-01,1,4,jmidb-bucket/accept_eath/kinki/2024-11-06/2024...,2024-11-06,False,False,False
909432,30,和歌山県,短期滞在手術等基本料１,2510479,NaN,東医1047,串本リハビリテーションセンター,649－3503,東牟婁郡串本町串本２５９－６,0735-62-3600,...,NaN,2024-11-01,2024-11-01,1,<NA>,jmidb-bucket/accept_eath/kinki/2024-11-06/2024...,2024-11-06,False,False,False


CPU times: user 39.6 s, sys: 1.36 s, total: 40.9 s
Wall time: 41.7 s


In [33]:
def drop_duplicate_base_date(df:pd.DataFrame) -> pd.DataFrame:
    """
    基準日の重複を削除する。
    データ取得日が最新のものを残す。
    """
    df = df.copy()
    group_columns = ['都道府県コード', '点数区分コード', '医療機関番号', '受理届出名称', '基準日']
    ids = df.groupby(group_columns)['データ取得日'].transform('max') == df['データ取得日']
    df = df[ids]
    df.reset_index(drop=True, inplace=True)
    return df

output_df = drop_duplicate_base_date(output_df)
print_dataframe_info(output_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 769340 entries, 0 to 769339
Data columns (total 24 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   都道府県コード           769340 non-null  Int64         
 1   都道府県名             769340 non-null  object        
 2   受理届出名称            769340 non-null  object        
 3   医療機関番号            769340 non-null  Int64         
 4   併設医療機関番号          68455 non-null   object        
 5   医療機関記号番号          164679 non-null  object        
 6   医療機関名称            769340 non-null  object        
 7   医療機関所在地（郵便番号）     769340 non-null  object        
 8   医療機関所在地（住所）       769340 non-null  object        
 9   電話番号              769340 non-null  object        
 10  FAX番号             606396 non-null  object        
 11  受理記号              769340 non-null  object        
 12  受理番号              769340 non-null  object        
 13  算定開始年月日           769340 non-null  object        
 14  個別有効

,都道府県コード,都道府県名,受理届出名称,医療機関番号,併設医療機関番号,医療機関記号番号,医療機関名称,医療機関所在地（郵便番号）,医療機関所在地（住所）,電話番号,...,個別有効開始年月日,基準日,作成日,点数区分コード,ベッド数,ファイルパス,データ取得日,医師少数区域,医師少数スポット,別表第六の二厚労大臣が定める地域
0,23,愛知県,精神病棟入院基本料,100058,110053,千種5,愛知県精神医療センター,464－0031,名古屋市千種区徳川山町４－１－７,052-763-1511,...,NaN,2024-04-01,2024-05-15,1,145,jmidb-bucket/accept_eath/tokaihokuriku/2024-05...,2024-05-17,False,False,False
1,23,愛知県,特定機能病院入院基本料,101536,111531,千種153,愛知県がんセンター,464－0021,名古屋市千種区鹿子殿１－１,052-762-6111,...,NaN,2024-04-01,2024-05-15,1,492,jmidb-bucket/accept_eath/tokaihokuriku/2024-05...,2024-05-17,False,False,False
2,23,愛知県,一般病棟入院基本料,102088,NaN,千種208,医療法人吉田病院,464－0854,名古屋市千種区大久手町５－１９,052-741-4187,...,NaN,2024-04-01,2024-05-15,1,57,jmidb-bucket/accept_eath/tokaihokuriku/2024-05...,2024-05-17,False,False,False
3,23,愛知県,療養病棟入院基本料,102088,NaN,千種208,医療法人吉田病院,464－0854,名古屋市千種区大久手町５－１９,052-741-4187,...,NaN,2024-04-01,2024-05-15,1,51,jmidb-bucket/accept_eath/tokaihokuriku/2024-05...,2024-05-17,False,False,False
4,23,愛知県,一般病棟入院基本料,102377,NaN,千種237,医療法人蜂友会 はちや整形外科病院,464－0821,名古屋市千種区末盛通２－４,052-751-8188,...,NaN,2024-04-01,2024-05-15,1,52,jmidb-bucket/accept_eath/tokaihokuriku/2024-05...,2024-05-17,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
769335,30,和歌山県,短期滞在手術等基本料１,2310284,NaN,新医1028,さとう眼科,647－0045,新宮市井の沢１２－６,0735-28-0310,...,NaN,2024-11-01,2024-11-01,1,4,jmidb-bucket/accept_eath/kinki/2024-11-06/2024...,2024-11-06,False,False,False
769336,30,和歌山県,短期滞在手術等基本料１,2410803,NaN,西医1080,上富田クリニック,649－2105,西牟婁郡上富田町朝来１４０７－１,0739-47-1100,...,NaN,2024-11-01,2024-11-01,1,<NA>,jmidb-bucket/accept_eath/kinki/2024-11-06/2024...,2024-11-06,False,False,False
769337,30,和歌山県,短期滞在手術等基本料１,2510461,NaN,東医1046,医療法人健佑会 串本有田病院,649－3514,東牟婁郡串本町有田４９９番地の１,0735-66-1021,...,NaN,2024-11-01,2024-11-01,1,4,jmidb-bucket/accept_eath/kinki/2024-11-06/2024...,2024-11-06,False,False,False
769338,30,和歌山県,短期滞在手術等基本料１,2510479,NaN,東医1047,串本リハビリテーションセンター,649－3503,東牟婁郡串本町串本２５９－６,0735-62-3600,...,NaN,2024-11-01,2024-11-01,1,<NA>,jmidb-bucket/accept_eath/kinki/2024-11-06/2024...,2024-11-06,False,False,False


In [34]:
sorted(output_df['受理届出名称'].unique())

['かかりつけ薬剤師指導料及びかかりつけ薬剤師包括管理料',
 'せん妄ハイリスク患者ケア加算',
 'コンタクトレンズ検査料１',
 'サーベイランス強化加算',
 'データ提出加算',
 'ニコチン依存症管理料',
 'ハイケアユニット入院医療管理料１',
 'ハイケアユニット入院医療管理料２',
 'ハイリスク分娩管理加算',
 'ハイリスク妊娠管理加算',
 'バイオ後続品使用体制加算',
 'リハビリテーション・栄養・口腔連携体制加算',
 '一般病棟入院基本料',
 '一類感染症患者入院医療管理料',
 '依存症入院医療管理加算',
 '児童・思春期精神科入院医療管理料',
 '入退院支援加算',
 '入院ベースアップ評価料１０３',
 '入院ベースアップ評価料１０４',
 '入院ベースアップ評価料１０５',
 '入院ベースアップ評価料１０６',
 '入院ベースアップ評価料１０８',
 '入院ベースアップ評価料１１',
 '入院ベースアップ評価料１１０',
 '入院ベースアップ評価料１１４',
 '入院ベースアップ評価料１１５',
 '入院ベースアップ評価料１１７',
 '入院ベースアップ評価料１１８',
 '入院ベースアップ評価料１２',
 '入院ベースアップ評価料１２０',
 '入院ベースアップ評価料１２４',
 '入院ベースアップ評価料１２６',
 '入院ベースアップ評価料１２７',
 '入院ベースアップ評価料１３',
 '入院ベースアップ評価料１３７',
 '入院ベースアップ評価料１３９',
 '入院ベースアップ評価料１４',
 '入院ベースアップ評価料１５',
 '入院ベースアップ評価料１５５',
 '入院ベースアップ評価料１５６',
 '入院ベースアップ評価料１６',
 '入院ベースアップ評価料１６５',
 '入院ベースアップ評価料１７',
 '入院ベースアップ評価料１８',
 '入院ベースアップ評価料１９',
 '入院ベースアップ評価料２',
 '入院ベースアップ評価料２０',
 '入院ベースアップ評価料２１',
 '入院ベースアップ評価料２２',
 '入院ベースアップ評価料２３',
 '入院ベースアップ評価料２４',
 '入院ベースアップ評価料２５',
 '入院ベースアップ評価料２６',
 '入院ベースアップ評価料２７',
 '入院ベース

In [35]:
output_df['医療機関コード'] = output_df['都道府県コード'].astype(str).str.zfill(2)\
                              + output_df['点数区分コード'].astype(str).str.zfill(1)\
                              + output_df['医療機関番号'].astype(str).str.zfill(7)

fileter = [
 '救命救急入院料１',
 '救命救急入院料２',
 '救命救急入院料３',
 '救命救急入院料４',
 '特定集中治療室管理料１',
 '特定集中治療室管理料２',
 '特定集中治療室管理料３',
 '特定集中治療室管理料４',
 '特定集中治療室管理料５',
 '特定集中治療室管理料６',
 '小児特定集中治療室管理料',
 'ハイケアユニット入院医療管理料１',
 'ハイケアユニット入院医療管理料２',
 '脳卒中ケアユニット入院医療管理料'
]
output_df = output_df[output_df['受理届出名称'].isin(fileter)]

print_dataframe_info(output_df)

<class 'pandas.core.frame.DataFrame'>
Index: 10833 entries, 1125 to 766470
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   都道府県コード           10833 non-null  Int64         
 1   都道府県名             10833 non-null  object        
 2   受理届出名称            10833 non-null  object        
 3   医療機関番号            10833 non-null  Int64         
 4   併設医療機関番号          6326 non-null   object        
 5   医療機関記号番号          2863 non-null   object        
 6   医療機関名称            10833 non-null  object        
 7   医療機関所在地（郵便番号）     10833 non-null  object        
 8   医療機関所在地（住所）       10833 non-null  object        
 9   電話番号              10833 non-null  object        
 10  FAX番号             8309 non-null   object        
 11  受理記号              10833 non-null  object        
 12  受理番号              10833 non-null  object        
 13  算定開始年月日           10833 non-null  object        
 14  個別有効開始年月日         0 non

,都道府県コード,都道府県名,受理届出名称,医療機関番号,併設医療機関番号,医療機関記号番号,医療機関名称,医療機関所在地（郵便番号）,医療機関所在地（住所）,電話番号,...,基準日,作成日,点数区分コード,ベッド数,ファイルパス,データ取得日,医師少数区域,医師少数スポット,別表第六の二厚労大臣が定める地域,医療機関コード
1125,23,愛知県,特定集中治療室管理料３,101536,111531,千種153,愛知県がんセンター,464－0021,名古屋市千種区鹿子殿１－１,052-762-6111,...,2024-04-01,2024-05-15,1,8,jmidb-bucket/accept_eath/tokaihokuriku/2024-05...,2024-05-17,False,False,False,2310101536
1126,23,愛知県,救命救急入院料１,105552,115557,千種555,名古屋市立大学医学部附属東部医療センター,464－8547,名古屋市千種区若水１－２－２３,052-721-7171,...,2024-04-01,2024-05-15,1,16,jmidb-bucket/accept_eath/tokaihokuriku/2024-05...,2024-05-17,False,False,False,2310105552
1127,23,愛知県,特定集中治療室管理料１,105552,115557,千種555,名古屋市立大学医学部附属東部医療センター,464－8547,名古屋市千種区若水１－２－２３,052-721-7171,...,2024-04-01,2024-05-15,1,10,jmidb-bucket/accept_eath/tokaihokuriku/2024-05...,2024-05-17,False,False,False,2310105552
1128,23,愛知県,ハイケアユニット入院医療管理料１,105552,115557,千種555,名古屋市立大学医学部附属東部医療センター,464－8547,名古屋市千種区若水１－２－２３,052-721-7171,...,2024-04-01,2024-05-15,1,4,jmidb-bucket/accept_eath/tokaihokuriku/2024-05...,2024-05-17,False,False,False,2310105552
1129,23,愛知県,ハイケアユニット入院医療管理料１,105552,115557,千種555,名古屋市立大学医学部附属東部医療センター,464－8547,名古屋市千種区若水１－２－２３,052-721-7171,...,2024-04-01,2024-05-15,1,6,jmidb-bucket/accept_eath/tokaihokuriku/2024-05...,2024-05-17,False,False,False,2310105552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
766386,30,和歌山県,特定集中治療室管理料５,8210595,8230599,田医1059,紀南病院,646－8588,田辺市新庄町４６番地７０,0739-22-5000,...,2024-11-01,2024-11-01,1,<NA>,jmidb-bucket/accept_eath/kinki/2024-11-06/2024...,2024-11-06,False,True,False,3018210595
766387,30,和歌山県,救命救急入院料１,8410377,8430371,田医1037,独立行政法人国立病院機構南和歌山医療センター,646－0015,田辺市たきない町２７－１,0739-26-7050,...,2024-11-01,2024-11-01,1,16,jmidb-bucket/accept_eath/kinki/2024-11-06/2024...,2024-11-06,False,True,False,3018410377
766388,30,和歌山県,特定集中治療室管理料６,8410377,8430371,田医1037,独立行政法人国立病院機構南和歌山医療センター,646－0015,田辺市たきない町２７－１,0739-26-7050,...,2024-11-01,2024-11-01,1,<NA>,jmidb-bucket/accept_eath/kinki/2024-11-06/2024...,2024-11-06,False,True,False,3018410377
766389,30,和歌山県,ハイケアユニット入院医療管理料１,8410377,8430371,田医1037,独立行政法人国立病院機構南和歌山医療センター,646－0015,田辺市たきない町２７－１,0739-26-7050,...,2024-11-01,2024-11-01,1,4,jmidb-bucket/accept_eath/kinki/2024-11-06/2024...,2024-11-06,False,True,False,3018410377


In [36]:
# 作成日ごとにシートを分けてExcelファイルに抽出
output_excel_path = 'output_df.xlsx'

select_columns = [
    '医療機関コード',
    '都道府県コード',
    '都道府県名',
    '医療機関名称',
    '受理届出名称',
    '受理記号',
    '医療機関所在地（郵便番号）',
    '医療機関所在地（住所）',
    '電話番号',
    '算定開始年月日',
    'ベッド数',
    '医師少数区域',
    '医師少数スポット',
    '別表第六の二厚労大臣が定める地域',
    'データ取得日',
    '基準日'
]
df = output_df.copy()

df = df[select_columns]
df = df.sort_values(by=['医療機関コード', '受理届出名称', 'データ取得日'])

bool_cols = ['医師少数区域', '医師少数スポット', '別表第六の二厚労大臣が定める地域']
for bool_col in bool_cols:
    df[bool_col] = df[bool_col].astype(str)
    df.loc[:, bool_col] = df[bool_col].replace({'True': '○', 'False': None})

df['基準日'] = df['基準日'].dt.strftime('%Y-%m-%d')
df['データ取得日'] = df['データ取得日'].dt.strftime('%Y-%m-%d')

with pd.ExcelWriter(output_excel_path) as writer:
    for date, date_df in df.groupby('基準日'):
        date_df.to_excel(writer, sheet_name=date, index=False)

In [37]:
output_df

,都道府県コード,都道府県名,受理届出名称,医療機関番号,併設医療機関番号,医療機関記号番号,医療機関名称,医療機関所在地（郵便番号）,医療機関所在地（住所）,電話番号,...,基準日,作成日,点数区分コード,ベッド数,ファイルパス,データ取得日,医師少数区域,医師少数スポット,別表第六の二厚労大臣が定める地域,医療機関コード
1125,23,愛知県,特定集中治療室管理料３,101536,111531,千種153,愛知県がんセンター,464－0021,名古屋市千種区鹿子殿１－１,052-762-6111,...,2024-04-01,2024-05-15,1,8,jmidb-bucket/accept_eath/tokaihokuriku/2024-05...,2024-05-17,False,False,False,2310101536
1126,23,愛知県,救命救急入院料１,105552,115557,千種555,名古屋市立大学医学部附属東部医療センター,464－8547,名古屋市千種区若水１－２－２３,052-721-7171,...,2024-04-01,2024-05-15,1,16,jmidb-bucket/accept_eath/tokaihokuriku/2024-05...,2024-05-17,False,False,False,2310105552
1127,23,愛知県,特定集中治療室管理料１,105552,115557,千種555,名古屋市立大学医学部附属東部医療センター,464－8547,名古屋市千種区若水１－２－２３,052-721-7171,...,2024-04-01,2024-05-15,1,10,jmidb-bucket/accept_eath/tokaihokuriku/2024-05...,2024-05-17,False,False,False,2310105552
1128,23,愛知県,ハイケアユニット入院医療管理料１,105552,115557,千種555,名古屋市立大学医学部附属東部医療センター,464－8547,名古屋市千種区若水１－２－２３,052-721-7171,...,2024-04-01,2024-05-15,1,4,jmidb-bucket/accept_eath/tokaihokuriku/2024-05...,2024-05-17,False,False,False,2310105552
1129,23,愛知県,ハイケアユニット入院医療管理料１,105552,115557,千種555,名古屋市立大学医学部附属東部医療センター,464－8547,名古屋市千種区若水１－２－２３,052-721-7171,...,2024-04-01,2024-05-15,1,6,jmidb-bucket/accept_eath/tokaihokuriku/2024-05...,2024-05-17,False,False,False,2310105552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
766386,30,和歌山県,特定集中治療室管理料５,8210595,8230599,田医1059,紀南病院,646－8588,田辺市新庄町４６番地７０,0739-22-5000,...,2024-11-01,2024-11-01,1,<NA>,jmidb-bucket/accept_eath/kinki/2024-11-06/2024...,2024-11-06,False,True,False,3018210595
766387,30,和歌山県,救命救急入院料１,8410377,8430371,田医1037,独立行政法人国立病院機構南和歌山医療センター,646－0015,田辺市たきない町２７－１,0739-26-7050,...,2024-11-01,2024-11-01,1,16,jmidb-bucket/accept_eath/kinki/2024-11-06/2024...,2024-11-06,False,True,False,3018410377
766388,30,和歌山県,特定集中治療室管理料６,8410377,8430371,田医1037,独立行政法人国立病院機構南和歌山医療センター,646－0015,田辺市たきない町２７－１,0739-26-7050,...,2024-11-01,2024-11-01,1,<NA>,jmidb-bucket/accept_eath/kinki/2024-11-06/2024...,2024-11-06,False,True,False,3018410377
766389,30,和歌山県,ハイケアユニット入院医療管理料１,8410377,8430371,田医1037,独立行政法人国立病院機構南和歌山医療センター,646－0015,田辺市たきない町２７－１,0739-26-7050,...,2024-11-01,2024-11-01,1,4,jmidb-bucket/accept_eath/kinki/2024-11-06/2024...,2024-11-06,False,True,False,3018410377


In [38]:
from google.colab import files
files.download(output_excel_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>